In [ ]:
#importing necessary libraries
from faker import Faker
import random
import mysql.connector

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 1.3 MB/s eta 0:00:02
   ---------------- ----------------------- 0.8/1.9 MB 1.3 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/1.9 MB 1.4 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/1.9 MB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
#Connect to the remote SQL database (TiDB Cloud)
conn = mysql.connector.connect(host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
                               user="4DTwJ2ZR7XTapss.root",
                               password="veNd2wXPKzLZwbmt",
                               port=4000)
cursor = conn.cursor()

In [ ]:
#creating a database and switching to it
cursor.execute('CREATE DATABASE IF NOT EXISTS Placement_Eligibility_App')
cursor.execute('USE Placement_Eligibility_App')

In [ ]:
# Class to handle student data generation and insertion into the 'students' table
class Students:
    def __init__(self, cursor):
        self.cursor = cursor
        self.faker = Faker("en_IN")
        self.year_list = [2022, 2023, 2024, 2025]
        self.gender_list = ["Female", "Male"]
        self.course_batch_list = [
            "DS-S-WE-N-B01", "DS-S-WD-N-B02", "DS-S-WE-N-B03", "DS-S-WD-N-B04",
            "AI-S-WE-N-B05", "AI-S-WD-N-B06", "AI-S-WE-N-B07", "AI-S-WD-N-B08",
            "FSD-S-WE-N-B09", "FSD-S-WD-N-B10", "FSD-S-WE-N-B11", "FSD-S-WD-N-B12",
            "AT-S-WE-N-B13", "AT-S-WD-N-B14", "AT-S-WE-N-B15", "AT-S-WE-N-B16"
        ]
# Create the 'students' table with relevant fields if it doesn't already exist
    def create_table(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS students (
                student_id INT PRIMARY KEY,
                name VARCHAR(50),
                age INT,
                gender VARCHAR(11),
                email VARCHAR(100),
                phone VARCHAR(13),
                enrollment_year INT,
                course_batch VARCHAR(20),
                city VARCHAR(50),
                graduation_year INT
            )
        """)
# Insert 500 synthetic student records
    def insert_data(self):
        for i in range(1, 501):
            student_id = i
            name = self.faker.name()
            email = self.faker.email()
            gender = random.choice(self.gender_list)
            age = random.randint(20, 45)
            city = self.faker.city()
            course_batch = random.choice(self.course_batch_list)
            enrollment_year = random.choice(self.year_list)
            graduation_year = enrollment_year + 3
            phone = '+91' + str(random.randint(6000000000, 9999999999))
# Insert the generated data into the 'students' table
            self.cursor.execute("""
                INSERT INTO students VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                student_id, name, age, gender, email, phone,
                enrollment_year, course_batch, city, graduation_year
            ))


In [ ]:
# Class to handle creation and data insertion for the Placement table
class Placement:
    def __init__(self, cursor):
        self.cursor = cursor
        self.faker = Faker("en_IN")
        self.placement_status_dict = {}

# Create the Placement table with relevant fields if it doesn't already exist
    def create_table(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS Placement (
                placement_id VARCHAR(5) PRIMARY KEY,
                student_id INT,
                mock_interview_score INT,
                internships_completed INT,
                placement_status VARCHAR(10),
                company_name VARCHAR(100),
                placement_package INT,
                interview_rounds_cleared INT,
                placement_date DATE
            )
        """)
# Insert 500 synthetic placcement records
    def insert_data(self):
        statuses = ["Placed", "Ready", "Not Ready"]

        for i in range(1, 501):
            student_id = i
            placement_id = f"PL{i:03d}"
            placement_status = random.choice(statuses)
            self.placement_status_dict[student_id] = placement_status

#Assign values based on placement_status
            if placement_status == "Placed":
                mock_interview_score = random.randint(80, 100)
                internships_completed = random.randint(2, 3)
                interview_rounds_cleared = random.randint(3, 5)
                company_name = self.faker.company()
                placement_package = random.randint(700000, 3000000)
                placement_date = self.faker.date_between(start_date='-3y', end_date='today')
            elif placement_status == "Ready":
                mock_interview_score = random.randint(50, 79)
                internships_completed = random.randint(1, 2)
                interview_rounds_cleared = random.randint(1, 3)
                company_name = ""
                placement_package = 0
                placement_date = None
            else:
                mock_interview_score = random.randint(30, 49)
                internships_completed = random.randint(0, 1)
                interview_rounds_cleared = random.randint(0, 1)
                company_name = ""
                placement_package = 0
                placement_date = None
                
# Insert the generated data into the Placement table
            self.cursor.execute("""
                INSERT INTO Placement VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                placement_id, student_id, mock_interview_score, internships_completed,
                placement_status, company_name, placement_package,
                interview_rounds_cleared, placement_date
            ))


In [ ]:
# Class to handle creation and data insertion for the Programming table
class Programming:
    def __init__(self, cursor, placement_status_dict):
        self.cursor = cursor
        self.placement_status_dict = placement_status_dict
        self.language_list = ['Python','SQL','JAVA','JAVASCRIPT','C++','R']

    def create_table(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS Programming (
                programming_id VARCHAR(5) PRIMARY KEY,
                student_id INT,
                language VARCHAR(50),
                problems_solved INT,
                assessments_completed INT,
                mini_projects INT,
                certifications_earned INT,
                latest_project_score INT,
                FOREIGN KEY (student_id) REFERENCES students(student_id)
            )
        """)
# Insert 500 synthetic Programming records
    def insert_data(self):
        for i in range(1, 501):
            status = self.placement_status_dict.get(i, "Not Ready")
            student_id = i
            programming_id = f"P{i:03d}"
            language = random.choice(self.language_list)
#assigning values based on placement_status
            if status == "Placed":
                problems_solved = random.randint(81, 120)
                assessments_completed = random.randint(8, 10)
                mini_projects = random.randint(3, 5)
                certifications_earned = random.randint(2, 4)
                latest_project_score = random.randint(85, 100)
            elif status == "Ready":
                problems_solved = random.randint(50, 80)
                assessments_completed = random.randint(5, 7)
                mini_projects = random.randint(2, 3)
                certifications_earned = random.randint(1, 2)
                latest_project_score = random.randint(70, 85)
            else:
                problems_solved = random.randint(10, 49)
                assessments_completed = random.randint(0, 4)
                mini_projects = random.randint(0, 1)
                certifications_earned = random.randint(0, 1)
                latest_project_score = random.randint(40, 70)
# Insert the generated data into the 'programming' table
            self.cursor.execute("""
                INSERT INTO Programming VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                programming_id, student_id, language, problems_solved,
                assessments_completed, mini_projects, certifications_earned, latest_project_score
            ))


In [ ]:
# Class to handle creation and data insertion for the softskills table
class SoftSkills:
    def __init__(self, cursor, placement_status_dict):
        self.cursor = cursor
        self.faker = Faker("en_IN")
        self.placement_status_dict = placement_status_dict
 # Create the SoftSkills table if it doesn't already exist
    def create_table(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS SoftSkills (
                soft_skill_id VARCHAR(5) PRIMARY KEY,
                student_id INT,
                communication INT,
                teamwork INT,
                presentation INT,
                leadership INT,
                critical_thinking INT,
                interpersonal_skills INT,
                soft_skill_avg FLOAT,
                FOREIGN KEY (student_id) REFERENCES students(student_id)
            )
        """)
# Insert 500 synthetic softskills records
    def insert_data(self):
        for i in range(1, 501):
            status = self.placement_status_dict.get(i, "Not Ready")
            soft_skill_id = f"S{i:03d}"
            student_id = i
# Assign soft skill scores based on placement status
            if status == "Placed":
                communication = random.randint(75, 100)
                teamwork = random.randint(75, 100)
                presentation = random.randint(75, 100)
                leadership = random.randint(75, 100)
                critical_thinking = random.randint(75, 100)
                interpersonal_skills = random.randint(75, 100)

            elif status == "Ready":
                communication = random.randint(55,74)
                teamwork = random.randint(55,74)
                presentation = random.randint(55,74)
                leadership = random.randint(55,74)
                critical_thinking = random.randint(55,74)
                interpersonal_skills = random.randint(55,74)
            else:
                communication = random.randint(20,54)
                teamwork = random.randint(20,54)
                presentation = random.randint(20,54)
                leadership = random.randint(20,54)
                critical_thinking = random.randint(20,54)
                interpersonal_skills = random.randint(20,54)

# Calculate average
            soft_skill_avg = round((
                communication + teamwork + presentation +
                leadership + critical_thinking + interpersonal_skills
            ) / 6, 2)
# Insert the record into the SoftSkills table
            self.cursor.execute("""
                INSERT INTO SoftSkills VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (soft_skill_id, student_id,communication, teamwork, presentation,leadership, critical_thinking, interpersonal_skills,soft_skill_avg
            ))

In [ ]:
#Create and populate the Students table
students = Students(cursor)
students.create_table()
students.insert_data()

#Create and populate the Placement table
placements = Placement(cursor)
placements.create_table()
placements.insert_data()

#Create and populate the programming table
programming = Programming(cursor, placements.placement_status_dict)
programming.create_table()
programming.insert_data()

#Create and populate the softskills table
softskills = SoftSkills(cursor, placements.placement_status_dict)
softskills.create_table()
softskills.insert_data()

#commit all the changes to the db
conn.commit()

#close the db
cursor.close()
conn.close()